In [1]:
from dataset_tf import VoxDataset
#import torch
#from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm as tqdm
from joblib import Parallel, delayed

In [2]:
dataset = VoxDataset()

In [3]:
dataset[0][1].shape

2024-03-16 21:36:36.398729: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-03-16 21:36:36.398751: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-16 21:36:36.398755: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-16 21:36:36.398780: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-16 21:36:36.398797: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


TensorShape([15069])

In [4]:
np.max(dataset[0][1])

270.77847

In [10]:
max = 0

for i in tqdm(dataset):
    curr_max = np.max(i[1])
    if curr_max > max:
        max = curr_max

100%|██████████| 107640/107640 [01:58<00:00, 908.41it/s]


In [11]:
max

279.79984

In [12]:
max = 0


for i in tqdm(dataset):
    curr_max = np.max(i[0])
    if curr_max > max:
        max = curr_max

100%|██████████| 107640/107640 [02:20<00:00, 763.49it/s]


In [13]:
max

226.92715